# Apache bean solution for data exploring

### Creado por: Yohan Alfonso Hernandez
### Fecha: 26-05-2023

Lest explore a clean data using the file "Clean_df" used in the project 'https://github.com/yohan-alfonso/Exploracion_Datos' the diference is thta in this project we are not gointo to plot just make reading and transforming since the propouse is to practice a little bit of Beam proraming

- In this part we are goin to create the reading and transformations of the dataset
- We are goin to sent the out put to a file or you can print it, if you are going to pirnt the resulta you have to uncomment the line

**Create Functions**

In [3]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText

class split_and_lowercase(beam.DoFn):
    def process(self, df):
        # Split the element into individual words
        words = df.split(',')
        # Convert each word to lowercase
        lowercased_words = [word.lower() for word in words]
        # Return the lowercase words as a list
        return [lowercased_words]
    
class filter_mean_salary(beam.DoFn):
    def process(self, column):
        column_value = column[3]
        if column_value != '0.0':
            return [column]
    


def run_pipeline(input_files, output_file):
    # Create the Pipeline object
    with beam.Pipeline() as pipeline:
        # Read the input files into a PCollection
        lines = pipeline | "ReadFiles" >> ReadFromText(input_files)
        
         # Split each line by commas
         # Split each line into words and convert to lowercase
        lowercased_words = lines | "SplitAndLowercase" >> beam.ParDo(split_and_lowercase())
        
        # filter column mean salary diferent from the string '0.0'
        filter_mean_sal = lowercased_words | "filter column" >> beam.ParDo(filter_mean_salary()) 
        
        # Write the split and filtered lines to the output file
        split_and_filtered_lines = filter_mean_sal | "WriteToFile" >> WriteToText(output_file, file_name_suffix='.csv')
     
        #split_and_filtered_lines | beam.Map(print)

# Specify the input and output files
input_files = 'clean_df.csv'
output_file = 'clean_df_filterd'

# Run the pipeline
run_pipeline(input_files, output_file)